In [ ]:
# The code is developed for the paper "Customer Segmentation Applied On RFM-ID Model"

# Imports

In [ ]:
import pandas as pd

# Loading Data 

## Data Description

InvoiceNo(Nominal): A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation. 

StockCode(Nominal): Product (item) code. A 5-digit integral number uniquely assigned to each distinct product. 

Description(String):  Product (item) name. 

Quantity(Numeric): The quantities of each product (item) per transaction.

InvoiceDate(Numeric): Invoice date and time. The day and time when a transaction was generated. 

UnitPrice(Numeric): Product price per unit in sterling (Â£). 

CustomerID(Nominal): A 5-digit integral number uniquely assigned to each customer. 

Country(Nominal): The name of the country where a customer resides.



In [ ]:
df = pd.read_excel('./data/online_retail.xlsx')

# Pre-processing

# RFM-ID Score

R(Recenecy): How recently did the customer purchase?

F(Frequency): How often do they purchase?

M(Monetary): How much do they spend?

I(Intenisty): How many do they buy per transaction?

D(Diversity): How many products do they buy?

# Segmentation Algorithms

# Comparison (Accuracy scores)